In [13]:
%run functions.py

In [82]:
table = 'contract-explorer-233919.ethparis.functions3'
limit = 'LIMIT 2000'

df = load_from_bigquery(table, limit)

In [83]:
target = df.hash
trees = df.tree
pp = Preprocess().fit_transform(trees)
X_train, X_test, y_train, y_test = train_test_split(pp, target, test_size=0.33,random_state=42)

In [84]:
pp = Preprocess()
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000)
toArray = ToArray()
pca = PCA(n_components=10)

pipe = Pipeline([('vectorizer', vectorizer ),
                ('toArray', toArray),
                 ('pca', pca)])


In [85]:
result = pipe.fit_transform(X_train)

In [86]:
rf = RandomForestClassifier(n_estimators=50,n_jobs=-1).fit(result,y_train)

In [87]:
###run pipeline for X_test
result_test = pipe.transform(X_test)

In [88]:
rf.score(result_test, y_test)

0.2287878787878788

In [89]:
#dumb model - predict most common class
target.value_counts()[0]/len(target)

0.03

In [90]:
#number of unique function hashes
len(y_train.unique())

940

In [108]:
n=3
probs = rf.predict_proba(result_test)

indices = np.argsort(probs, axis=1)[:,-n:]

top_n_classes = rf.classes_[indices]

p = copy(probs)
p.sort(axis=1)
top_n_probs = p[:,-n:]

In [117]:
top_n_classes[0:10][:,::-1]

array([['0x42966c68', '0x6623fc46', '0x3176d373'],
       ['0x439455e4', '0x9caa9448', '0xf46a85a9'],
       ['0x5c658165', '_fallback()', '0x58004a1d'],
       ['0xf9d176b4', '0x1608f18f', '0x1a81af29'],
       ['0x78bd7935', '0xbae804a7', '0x675518b8'],
       ['0xa0e67e2b', '0x259bc6f5', '0x9b487f3f'],
       ['0x42966c68', '0x1c223e2e', '0x97304ced'],
       ['0x1a44731c', '0xaaa2b8c8', '0x5d6af2af'],
       ['0x1785f53c', '0x45c8b1a6', '0x4b96b0ee'],
       ['0x0988ca8c', '0x0913348f', '0x4eee966f']], dtype=object)

In [118]:
top_n_probs[0:10][:,::-1]

array([[0.6       , 0.04      , 0.04      ],
       [0.42      , 0.16      , 0.04      ],
       [1.        , 0.        , 0.        ],
       [0.08      , 0.06      , 0.06      ],
       [0.1       , 0.08      , 0.06      ],
       [0.54      , 0.1       , 0.04      ],
       [0.16      , 0.1       , 0.06      ],
       [0.27569841, 0.23963492, 0.23715873],
       [0.1       , 0.06      , 0.06      ],
       [0.34      , 0.16      , 0.12      ]])

In [119]:
y_test[0:10]

1860    0x42966c68
353     0xc825798a
1333    0x5c658165
905     0x02329a29
1289    0x82771c8e
1273    0x3f67a94e
938     0x4b1fb6d4
1731    0x8da5cb5b
65      0x70480275
1323    0x0988ca8c
Name: hash, dtype: object

In [124]:
#pickle
pickle.dump(rf, open( "pickle/rf.p", "wb" ) )
pickle.dump(pipe, open( "pickle/pipe.p", "wb" ) )